<a href="https://colab.research.google.com/github/boscoybarra/customer-transaction-prediction/blob/master/customer_transaction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading Kaggle Dataset and dependencies


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

In [0]:
# Install kaggle pip
!pip install -U -q kaggle

!mkdir ~/.kaggle/

# We copy the json file into a apparently temp file .kaggle
!cp "/content/drive/My Drive/kaggle/kaggle.json" ~/.kaggle/




    100% |████████████████████████████████| 61kB 3.7MB/s 


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

##Download the data 

In [0]:
!kaggle competitions download santander-customer-transaction-prediction
!ls



 92% 113M/122M [00:00<00:00, 103MB/s]  
100% 122M/122M [00:00<00:00, 136MB/s]
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 78.4MB/s]
 87% 106M/122M [00:00<00:00, 96.7MB/s] 
100% 122M/122M [00:01<00:00, 125MB/s] 
drive  sample_data  sample_submission.csv.zip  test.csv.zip  train.csv.zip


##Unzip the Data

In [0]:
# import pandas as pd
# train = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
# test = pd.read_csv('test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
# sample_submission = pd.read_csv('/content/sample_submission.csv.zip', compression='zip', header=0, sep=',', quotechar='"')



#Starting point


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics

In [0]:
train = pd.read_csv('train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
test = pd.read_csv('test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
targets = train[train.columns[1:2]]
# train = train[train.columns[2:]]
# test = test[test.columns[1:]]
targets[1:]


,target
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
10,0


In [0]:
target_transactions = targets.sum(axis=None)
print(target_transactions)

targets.shape

target    20098
dtype: int64


(200000, 1)

#### TODO 0: get all values that have a target of 1 and try to generate a regression for all 200 variables


#### TODO 1: train with 20098 taregets vs another 20098 random 0 targets ?



In [0]:
features

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

In [0]:
train_test = pd.concat([train, test], axis =0)



target = train_test['target'].values

test_target = train['target']


In [0]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 17,
         "metric": 'auc',
         "verbosity": -1}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train_test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_test.values, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_test.iloc[trn_idx][features], label=target[trn_idx])
    val_data = lgb.Dataset(train_test.iloc[val_idx][features], label=target[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_test.iloc[val_idx][features], num_iteration=clf.best_iteration)

fold n°0
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.633934	valid_1's auc: 0.511805
[200]	training's auc: 0.680832	valid_1's auc: 0.512238
Early stopping, best iteration is:
[128]	training's auc: 0.649589	valid_1's auc: 0.512908
fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.635347	valid_1's auc: 0.511121
[200]	training's auc: 0.681405	valid_1's auc: 0.514333
[300]	training's auc: 0.713812	valid_1's auc: 0.515166


KeyboardInterrupt: ignored

Let's take a look at the overall oof CV AUC:

In [0]:
metrics.roc_auc_score(target, oof)

AUC of 0.52 is overall not a major issue, but it is slightly over 0.50, which can have an impact when you are chasing the 4th decimal point.

## Lasso Implementation 
https://www.kaggle.com/baogorek/lasso-regression-with-keras